In [2]:
import pandas as pd
import json

In [3]:
json1_file = open('./CleanedData/Council_to_Code.json')
json1_str = json1_file.read()
code_dict = json.loads(json1_str)

In [4]:
df_tercile = pd.read_csv("./CleanedData/Income_Tercile_Map.csv")
df_tercile = df_tercile.tail(-1)
df_tercile.rename(columns = {"Year": "Council Code"}, inplace=True)
df_tercile

,Council Code,2016,2017,2018,2019,2020,2021,2022,2023,2024,2025
1,Aberdeen City,High,High,High,High,High,High,High,High,High,High
2,Aberdeenshire,Low,Low,Mid,Mid,Mid,Mid,Mid,Mid,Mid,Mid
3,Adur,High,Mid,Mid,Low,Low,Low,Low,Mid,Low,Low
4,Amber Valley,High,High,Mid,High,Mid,Low,Low,Low,Mid,Mid
5,Angus,Low,Low,Low,High,Low,Low,Mid,Low,Mid,Low
...,...,...,...,...,...,...,...,...,...,...,...
374,Wychavon,Mid,Mid,Mid,Mid,Low,Mid,High,High,High,Mid
375,Wyre,Low,High,Low,Low,Low,Low,Low,Low,Low,Low
376,Wyre Forest,High,Low,Low,Low,Low,Low,Low,High,Low,Low
377,York UA,Mid,Mid,Low,Mid,Mid,Mid,High,Mid,High,Mid


In [ ]:
tercile = dict(zip(df_tercile["Council Code"], df_tercile["2016"]))


,Origin,Destination,0-16,17-34,35-54,55-66,67+
0,High,Adur,94.1347,276.5311,154.6864,68.9276,637.9399
320,Low,Adur,191.3332,384.9149,233.3817,75.7254,951.3196
640,Mid,Adur,300.6216,694.6481,593.9238,173.6040,1948.6657
1,High,Amber Valley,287.9745,733.7648,509.9332,166.8197,1783.8266
321,Low,Amber Valley,201.6964,512.6190,344.7607,164.7060,1393.2904
...,...,...,...,...,...,...,...
318,High,Wyre Forest,150.6929,449.9186,294.5938,115.3423,1113.8795
638,Low,Wyre Forest,279.6218,520.9492,497.4712,221.8051,1688.2674
639,Low,York UA,221.0083,2831.8546,422.5193,137.2984,3768.9012
319,High,York UA,264.5170,3282.7320,562.8302,142.4107,4358.4596


In [ ]:
for year in range (2016, 2025):
    if (year < 2022):
        df= pd.read_csv(f"./Temp/Migration_{year}.csv")
    else:
        df= pd.read_excel(f"./Data/Migration/Migration_{year}.xlsx")
    tercile = dict(zip(df_tercile["Council Code"], df_tercile[f"{year}"]))
    df.rename(columns={'outla': "Origin"}, inplace=True)
    df.rename(columns={'inla': "Destination"}, inplace=True)
    if (year < 2022):
        df.drop(columns=["Year", "sex"], axis=1, inplace=True)
    else:
        df.drop(columns=["year", "sex"], axis=1, inplace=True)
    df.replace({"Destination": code_dict}, inplace=True)
    df.replace({"Origin": code_dict}, inplace=True)
    df["Origin"] = df["Origin"].str.strip()
    df["Destination"] = df["Destination"].str.strip()
    df.replace({"Origin": tercile}, inplace=True)
    df["0-16"] = df.loc[:, "Age_0":"Age_16"].sum(axis=1)
    df["17-34"] = df.loc[:, "Age_17":"Age_34"].sum(axis=1)
    df["35-54"] = df.loc[:, "Age_35":"Age_54"].sum(axis=1)
    df["55-66"] = df.loc[:, "Age_55":"Age_66"].sum(axis=1)
    df["67+"] = df.loc[:, "Age_67":].sum(axis=1)
    df.drop(columns=[c for c in df.columns if c not in ["Origin", "Destination", "0-16", "17-34", "35-54", "55-66", "67+"]], inplace=True)
    df = df[["Destination", "Origin", '0-16', "17-34", "35-54", "55-66", "67+"]] 
    df.sort_values(by="Destination", inplace=True)
    df = df[df["Origin"].isin(["Mid", "Low", "High"])]
    df = df.groupby(["Origin", "Destination"], as_index=False).sum()
    df.sort_values(by="Destination", inplace=True)
    df.to_csv(f"./Cleaned_Data/Migration_Cleaned_{year}.csv", index=True)  